In [6]:
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_package("langchain-openai")
import json
import os
import time
import hashlib
from typing import List, Dict, Optional, TypedDict, Union, Any
from datetime import datetime
from contextlib import contextmanager

# Универсальный импорт Pydantic с поддержкой V1 и V2
try:
    # Пытаемся импортировать V2 API
    from pydantic import BaseModel, Field, field_validator, ConfigDict
    PYDANTIC_V2 = True
except ImportError:
    # Fallback для V1
    from pydantic import BaseModel, Field, validator
    PYDANTIC_V2 = False

from langgraph.graph import StateGraph, END

# --- Начало: Устойчивый импорт для MemorySaver ---
try:
    # Попытка 1: Стандартный импорт для последних версий LangGraph
    from langgraph.checkpoint.memory import MemorySaver
    CheckpointSaver = MemorySaver
except ImportError:
    try:
        # Попытка 2: Альтернативный импорт для других версий
        from langgraph.checkpoint.memory import InMemorySaver
        CheckpointSaver = InMemorySaver
        print("INFO: Используется InMemorySaver как резервный вариант.")
    except ImportError as e:
        raise ImportError(
            "Не удалось импортировать MemorySaver или InMemorySaver из 'langgraph.checkpoint.memory'. "
            "Убедитесь, что langgraph установлен правильно. Попробуйте: pip install -U langgraph"
        ) from e
# --- Конец: Устойчивый импорт для MemorySaver ---

# Импорт LangSmith с обработкой ошибок
try:
    from langsmith import Client, traceable, RunTree
    LANGCHAIN_SMITH_AVAILABLE = True
except ImportError:
    LANGCHAIN_SMITH_AVAILABLE = False
    print("WARNING: LangSmith не установлен. Трассировка отключена.")
    
from langchain_core.runnables import RunnableConfig

# Импорт типа Message из telethon
try:
    from telethon.tl.types import Message as TelegramMessage
except ImportError:
    TelegramMessage = None

# Настройка LangSmith
if LANGCHAIN_SMITH_AVAILABLE and os.environ.get("LANGSMITH_API_KEY"):
    client = Client()
else:
    client = None


class Event(BaseModel):
    """Модель события для структурированного извлечения"""
    title: Optional[str] = Field(None, description="Название события")
    description: Optional[str] = Field(None, description="Описание события")
    event_type: Optional[str] = Field(None, description="Тип события: лекция, встреча, семинар и т.д.")
    is_online: Optional[bool] = Field(None, description="True - онлайн, False - офлайн, None - неизвестно")
    location: Optional[str] = Field(None, description="Место проведения или ссылка")
    date: Optional[str] = Field(None, description="Дата события в формате YYYY-MM-DD")
    time: Optional[str] = Field(None, description="Время события в формате HH:MM")
    source_message_id: Optional[int] = Field(None, description="ID исходного сообщения")
    original_text: Optional[str] = Field(None, description="Оригинальный текст")
    # Дополнительные поля для обогащения
    processed_at: Optional[str] = Field(None, description="Временная метка обработки")
    category: Optional[str] = Field(None, description="Категория: education, meeting, workshop, other")

    # Конфигурация Pydantic с поддержкой V1 и V2
    if PYDANTIC_V2:
        model_config = ConfigDict(extra="forbid", arbitrary_types_allowed=True)
        
        # Валидаторы для Pydantic V2
        @field_validator('date')
        @classmethod
        def validate_date_v2(cls, v: Optional[str]) -> Optional[str]:
            if v:
                try:
                    datetime.strptime(v, '%Y-%m-%d')
                except ValueError:
                    raise ValueError("Дата должна быть в формате YYYY-MM-DD")
            return v
        
        @field_validator('time')
        @classmethod
        def validate_time_v2(cls, v: Optional[str]) -> Optional[str]:
            if v:
                try:
                    datetime.strptime(v, '%H:%M')
                except ValueError:
                    raise ValueError("Время должно быть в формате HH:MM")
            return v
    else:
        # Конфигурация для Pydantic V1
        class Config:
            extra = "forbid"
            arbitrary_types_allowed = True
        
        # Валидаторы для Pydantic V1
        @validator('date')
        def validate_date_v1(cls, v):
            if v:
                try:
                    datetime.strptime(v, '%Y-%m-%d')
                except ValueError:
                    raise ValueError("Дата должна быть в формате YYYY-MM-DD")
            return v
        
        @validator('time')
        def validate_time_v1(cls, v):
            if v:
                try:
                    datetime.strptime(v, '%H:%M')
                except ValueError:
                    raise ValueError("Время должно быть в формате HH:MM")
            return v

    @classmethod
    def parse_from_json(cls, json_string: str) -> List['Event']:
        """Парсинг событий из JSON строки"""
        try:
            parsed_data = json.loads(json_string)
        except json.JSONDecodeError as e:
            print(f"Ошибка парсинга JSON: {e}")
            return []

        if isinstance(parsed_data, dict):
            events_data = parsed_data.get('events', [])
        elif isinstance(parsed_data, list):
            events_data = parsed_data
        else:
            return []

        events = []
        for event_data in events_data:
            if isinstance(event_data, dict):
                try:
                    if hasattr(cls, 'model_validate'):
                        event = cls.model_validate(event_data)
                    else:
                        event = cls.parse_obj(event_data)
                    events.append(event)
                except Exception as e:
                    print(f"Ошибка валидации события: {e}")
                    continue
        return events

    def to_dict(self) -> Dict:
        """Конвертация в словарь"""
        if hasattr(self, 'model_dump'):
            return self.model_dump(exclude_none=True)
        else:
            return self.dict(exclude_none=True)


class EventsList(BaseModel):
    """Обертка для списка событий"""
    events: List[Event] = Field(default_factory=list)

    # Конфигурация Pydantic с поддержкой V1 и V2
    if PYDANTIC_V2:
        model_config = ConfigDict(extra="forbid")
    else:
        class Config:
            extra = "forbid"


class EventExtractor:
    """Класс для извлечения событий с поддержкой LangSmith трассировки"""

    def __init__(self, llm, model_name: str = "gpt-4o"):
        self.llm = llm
        self.model_name = model_name

    @staticmethod
    def _message_to_dict(message: Union[Dict, 'TelegramMessage']) -> Dict:
        """Конвертация сообщения в словарь"""
        # Декоратор @traceable условный
        if LANGCHAIN_SMITH_AVAILABLE:
            # Создаем условный декоратор
            def traceable_decorator(func):
                def wrapper(*args, **kwargs):
                    return func(*args, **kwargs)
                return wrapper
            _decorator = traceable_decorator
        else:
            _decorator = lambda x: x
        
        @_decorator
        def _inner_message_to_dict(message: Union[Dict, 'TelegramMessage']) -> Dict:
            if isinstance(message, dict):
                return message

            if TelegramMessage and isinstance(message, TelegramMessage):
                return {
                    'id': message.id,
                    'date': message.date.isoformat() if message.date else None,
                    'text': message.message or "",
                    'sender_id': message.sender_id if hasattr(message, 'sender_id') else None,
                    'chat_id': message.chat_id if hasattr(message, 'chat_id') else None
                }

            raise TypeError(f"Неподдерживаемый тип сообщения: {type(message)}")
        
        return _inner_message_to_dict(message)

    def extract_events(
            self,
            messages: List[Union[Dict, 'TelegramMessage']],
            config: Optional[RunnableConfig] = None
    ) -> List[Event]:
        """Основной метод извлечения событий"""
        if not messages:
            return []

        # Конвертируем сообщения в словари
        messages_dict = [self._message_to_dict(msg) for msg in messages]

        # Формируем текст сообщений для промпта
        messages_text = "\n\n".join([
            f"Сообщение #{i + 1} (ID: {msg['id']}, Дата: {msg['date']}):\n{msg['text'][:1000]}"
            for i, msg in enumerate(messages_dict)
        ])

        prompt = f"""Ты - эксперт по извлечению структурированной информации о событиях из текстовых сообщений.

Проанализируй следующие сообщения из Telegram и извлеки из них все упоминания о событиях (лекции, встречи, семинары, конференции, мастер-классы и т.д.).

Для каждого найденного события заполни все доступные поля:
1. title: краткое название события
2. description: детальное описание (если есть)
3. event_type: тип события (выбери из: лекция, встреча, семинар, конференция, мастер-класс, вебинар, экзамен, дедлайн, праздник, другое)
4. is_online: True - если указано, что событие онлайн; False - если офлайн; null - если неясно
5. location: конкретное место или ссылка для подключения
6. date: дата в формате YYYY-MM-DD
7. time: время в формате HH:MM
8. source_message_id: ID сообщения, из которого извлечено событие
9. original_text: точный фрагмент текста, содержащий информацию о событии

ВАЖНЫЕ ПРАВИЛА:
- Извлекай только реальные события с указанием времени/даты
- Если дата относительная ("завтра", "в следующую пятницу"), преобразуй в абсолютную
- Для онлайн-событий в location указывай ссылку или "онлайн"
- Если информация не найдена, оставляй поле null
- Не придумывай информацию, которой нет в тексте

Сообщения:
{messages_text}

Верни результат строго в формате JSON с ключом "events", содержащим массив объектов событий.
Если событий не найдено, верни {{"events": []}}."""

        try:
            # Трассируем вызов LLM через LangSmith, если доступен
            if LANGCHAIN_SMITH_AVAILABLE:
                with self._langsmith_trace("llm_inference", config) as run_tree:
                    return self._execute_extraction(prompt, messages_dict, run_tree, config)
            else:
                return self._execute_extraction(prompt, messages_dict, None, config)

        except Exception as e:
            print(f"Ошибка при извлечении событий: {e}")
            if client:
                client.create_feedback(
                    run_id=None,  # Будет заполнено LangSmith
                    key="extraction_error",
                    score=0.0,
                    comment=str(e)
                )
            return []

    def _execute_extraction(self, prompt: str, messages_dict: List[Dict], 
                           run_tree: Optional[Any], config: Optional[RunnableConfig]) -> List[Event]:
        """Выполнение извлечения событий"""
        if run_tree:
            run_tree.inputs = {"prompt": prompt[:500]}

        # Используем structured output от OpenAI
        if hasattr(self.llm, 'with_structured_output'):
            structured_llm = self.llm.with_structured_output(EventsList)
            response = structured_llm.invoke(prompt, config=config)
            events_list = response
        elif hasattr(self.llm, 'bind') and hasattr(self.llm, 'schema'):
            # Альтернативный способ для LangChain
            from langchain_core.output_parsers import JsonOutputParser
            from langchain_core.prompts import ChatPromptTemplate

            prompt_template = ChatPromptTemplate.from_template(prompt)
            parser = JsonOutputParser(pydantic_object=EventsList)

            chain = prompt_template | self.llm | parser
            events_list = chain.invoke({}, config=config)
        else:
            # Fallback для других LLM
            response = self.llm.invoke(prompt, config=config)
            if hasattr(response, 'content'):
                result_text = response.content
            else:
                result_text = str(response)

            result_text = self._clean_json_response(result_text)
            events = Event.parse_from_json(result_text)
            events_list = EventsList(events=events)

        events = events_list.events if events_list else []

        if run_tree:
            run_tree.outputs = {"events_count": len(events)}

        # Обогащаем события дополнительной информацией
        for idx, event in enumerate(events):
            if not event.source_message_id and idx < len(messages_dict):
                event.source_message_id = messages_dict[idx]['id']
            if not event.original_text and idx < len(messages_dict):
                # Берем только релевантную часть текста
                event.original_text = self._extract_relevant_text(
                    messages_dict[idx]['text'],
                    event
                )

        return events

    @contextmanager
    def _langsmith_trace(self, name: str, config: Optional[RunnableConfig] = None):
        """Контекстный менеджер для трассировки LangSmith"""
        if not client or not os.environ.get("LANGSMITH_TRACING", "true").lower() == "true":
            yield None
            return

        try:
            run_tree = RunTree(
                name=name,
                run_type="chain",
                inputs={},
                outputs={},
                serialized={},
                project_name=os.environ.get("LANGSMITH_PROJECT", "event-miner"),
                tags=["extraction", "telegram"],
            )

            if config and config.get("configurable", {}).get("thread_id"):
                run_tree.thread_id = config["configurable"]["thread_id"]

            run_tree.post()
            yield run_tree

            run_tree.end()
            run_tree.patch()

        except Exception as e:
            print(f"Ошибка трассировки LangSmith: {e}")
            yield None

    @staticmethod
    def _extract_relevant_text(full_text: str, event: Event) -> str:
        """Извлечение наиболее релевантного фрагмента текста"""
        if not event.title or len(full_text) < 200:
            return full_text[:500]

        # Ищем упоминание заголовка в тексте
        title_lower = event.title.lower()
        text_lower = full_text.lower()

        if title_lower in text_lower:
            idx = text_lower.find(title_lower)
            start = max(0, idx - 100)
            end = min(len(full_text), idx + len(event.title) + 200)
            return full_text[start:end]

        # Возвращаем начало текста, если заголовок не найден
        return full_text[:500]

    @staticmethod
    def _clean_json_response(text: str) -> str:
        """Очистка JSON ответа от лишних символов"""
        text = text.strip()

        # Удаляем markdown code blocks
        if text.startswith("```json"):
            text = text[7:]
        elif text.startswith("```"):
            text = text[3:]
        if text.endswith("```"):
            text = text[:-3]

        # Удаляем лишние пробелы и переносы
        text = text.strip()

        # Пытаемся найти JSON в тексте
        import re
        json_pattern = r'\{.*"events".*\]\}'
        match = re.search(json_pattern, text, re.DOTALL)
        if match:
            return match.group(0)

        return text


class AgentState(TypedDict):
    """Состояние агента LangGraph"""
    messages: List[Dict]
    events: List[Event]
    processed_count: int
    extraction_errors: List[str]
    metadata: Dict[str, Any]


class EventMinerAgent:
    """Основной агент для извлечения событий с полной оркестрацией LangGraph"""

    def __init__(
            self,
            llm,
            event_extractor: Optional[EventExtractor] = None,
            checkpoint_memory: bool = True
    ):
        self.llm = llm
        self.event_extractor = event_extractor or EventExtractor(llm)

        # Используем устойчивый импорт для создания checkpointer
        self.checkpointer = CheckpointSaver() if checkpoint_memory else None

        self.graph = self._build_graph()
        self._setup_langsmith()

    def _setup_langsmith(self):
        """Настройка LangSmith клиента"""
        if client and os.environ.get("LANGSMITH_API_KEY"):
            try:
                # Регистрируем проект
                project_name = os.environ.get("LANGSMITH_PROJECT", "event-miner-agent")
                client.create_project(
                    project_name=project_name,
                    description="Агент для извлечения событий из Telegram сообщений"
                )
                print(f"LangSmith проект '{project_name}' настроен")
            except Exception as e:
                print(f"Ошибка настройки LangSmith: {e}")

    def _build_graph(self) -> StateGraph:
        """Построение графа LangGraph"""
        workflow = StateGraph(AgentState)

        # Добавляем узлы
        workflow.add_node("preprocess_messages", self._preprocess_messages_node)
        workflow.add_node("extract_events", self._extract_events_node)
        workflow.add_node("validate_events", self._validate_events_node)
        workflow.add_node("enrich_events", self._enrich_events_node)
        workflow.add_node("handle_errors", self._handle_errors_node)

        # Определяем поток
        workflow.set_entry_point("preprocess_messages")

        # Основной поток
        workflow.add_edge("preprocess_messages", "extract_events")
        workflow.add_edge("extract_events", "validate_events")
        workflow.add_edge("validate_events", "enrich_events")
        workflow.add_conditional_edges(
            "enrich_events",
            self._should_handle_errors,
            {
                "continue": END,
                "handle_errors": "handle_errors"
            }
        )
        workflow.add_edge("handle_errors", END)

        # Добавляем механизм контрольных точек, если он был создан
        if self.checkpointer:
            return workflow.compile(checkpointer=self.checkpointer)
        return workflow.compile()

    def _preprocess_messages_node(self, state: AgentState) -> AgentState:
        """Предобработка сообщений"""
        # Декоратор @traceable условный
        if LANGCHAIN_SMITH_AVAILABLE:
            # Создаем условный декоратор
            def traceable_decorator(name, run_type):
                def decorator(func):
                    def wrapper(*args, **kwargs):
                        return func(*args, **kwargs)
                    return wrapper
                return decorator
            _decorator = traceable_decorator("preprocess_messages", "chain")
        else:
            _decorator = lambda x: x
        
        @_decorator
        def _inner_preprocess_messages(state: AgentState) -> AgentState:
            messages = state.get("messages", [])

            if not messages:
                return {
                    **state,
                    "extraction_errors": ["Нет сообщений для обработки"]
                }

            # Фильтруем пустые сообщения
            filtered_messages = [
                msg for msg in messages
                if msg.get('text') and len(msg.get('text', '').strip()) > 10
            ]

            return {
                **state,
                "messages": filtered_messages,
                "metadata": {
                    "total_messages": len(messages),
                    "filtered_messages": len(filtered_messages),
                    "filtered_out": len(messages) - len(filtered_messages)
                }
            }
        
        return _inner_preprocess_messages(state)

    def _extract_events_node(self, state: AgentState) -> AgentState:
        """Извлечение событий из сообщений"""
        if LANGCHAIN_SMITH_AVAILABLE:
            def traceable_decorator(name, run_type):
                def decorator(func):
                    def wrapper(*args, **kwargs):
                        return func(*args, **kwargs)
                    return wrapper
                return decorator
            _decorator = traceable_decorator("extract_events", "chain")
        else:
            _decorator = lambda x: x
        
        @_decorator
        def _inner_extract_events(state: AgentState) -> AgentState:
            messages = state.get("messages", [])

            if not messages:
                return {
                    **state,
                    "events": [],
                    "processed_count": 0
                }

            try:
                # Извлекаем события с помощью EventExtractor
                events = self.event_extractor.extract_events(messages)

                return {
                    **state,
                    "events": events,
                    "processed_count": len(messages)
                }
            except Exception as e:
                error_msg = f"Ошибка извлечения событий: {str(e)}"
                return {
                    **state,
                    "events": [],
                    "processed_count": 0,
                    "extraction_errors": [error_msg]
                }
        
        return _inner_extract_events(state)

    def _validate_events_node(self, state: AgentState) -> AgentState:
        """Валидация извлеченных событий"""
        if LANGCHAIN_SMITH_AVAILABLE:
            def traceable_decorator(name, run_type):
                def decorator(func):
                    def wrapper(*args, **kwargs):
                        return func(*args, **kwargs)
                    return wrapper
                return decorator
            _decorator = traceable_decorator("validate_events", "chain")
        else:
            _decorator = lambda x: x
        
        @_decorator
        def _inner_validate_events(state: AgentState) -> AgentState:
            events = state.get("events", [])
            errors = state.get("extraction_errors", [])

            validated_events = []
            for event in events:
                # Проверяем минимальные требования к событию
                if (event.title or event.description) and (event.date or event.time):
                    validated_events.append(event)
                elif event.title and event.event_type:
                    validated_events.append(event)
                else:
                    errors.append(f"Событие без минимальной информации: {event.title}")

            # Добавляем статистику
            metadata = state.get("metadata", {})
            metadata["validated_events"] = len(validated_events)
            metadata["rejected_events"] = len(events) - len(validated_events)

            return {
                **state,
                "events": validated_events,
                "extraction_errors": errors,
                "metadata": metadata
            }
        
        return _inner_validate_events(state)

    def _enrich_events_node(self, state: AgentState) -> AgentState:
        """Обогащение событий дополнительной информацией"""
        if LANGCHAIN_SMITH_AVAILABLE:
            def traceable_decorator(name, run_type):
                def decorator(func):
                    def wrapper(*args, **kwargs):
                        return func(*args, **kwargs)
                    return wrapper
                return decorator
            _decorator = traceable_decorator("enrich_events", "chain")
        else:
            _decorator = lambda x: x
        
        @_decorator
        def _inner_enrich_events(state: AgentState) -> AgentState:
            events = state.get("events", [])
            errors = state.get("extraction_errors", [])

            enriched_events = []
            for event in events:
                try:
                    # Добавляем временную метку обработки
                    event_dict = event.to_dict()
                    event_dict["processed_at"] = datetime.now().isoformat()

                    # Определяем категорию события на основе типа
                    event_type = event.event_type or ""
                    type_lower = event_type.lower()
                    if any(word in type_lower for word in ["лекция", "семинар", "вебинар"]):
                        event_dict["category"] = "education"
                    elif any(word in type_lower for word in ["встреча", "конференция"]):
                        event_dict["category"] = "meeting"
                    elif any(word in type_lower for word in ["мастер-класс", "воркшоп"]):
                        event_dict["category"] = "workshop"
                    else:
                        event_dict["category"] = "other"

                    # Создаем новый объект Event с дополнительными полями
                    enriched_event = Event(**event_dict)
                    enriched_events.append(enriched_event)

                except Exception as e:
                    errors.append(f"Ошибка обогащения события {event.title}: {str(e)}")
                    enriched_events.append(event)  # Оставляем исходное событие

            return {
                **state,
                "events": enriched_events,
                "extraction_errors": errors
            }
        
        return _inner_enrich_events(state)

    def _handle_errors_node(self, state: AgentState) -> AgentState:
        """Обработка ошибок"""
        errors = state.get("extraction_errors", [])

        if errors and client:
            # Логируем ошибки в LangSmith
            for error in errors:
                try:
                    client.create_feedback(
                        run_id=None,
                        key="processing_error",
                        score=0.0,
                        comment=error[:500]
                    )
                except Exception:
                    pass

        return state

    def _should_handle_errors(self, state: AgentState) -> str:
        """Определяет, нужно ли обрабатывать ошибки"""
        errors = state.get("extraction_errors", [])
        return "handle_errors" if errors else "continue"

    def process_messages(
            self,
            messages: List[Union[Dict, 'TelegramMessage']],
            config: Optional[RunnableConfig] = None
    ) -> List[Event]:
        """Основной метод обработки сообщений"""
        if LANGCHAIN_SMITH_AVAILABLE:
            def traceable_decorator(name, run_type):
                def decorator(func):
                    def wrapper(*args, **kwargs):
                        return func(*args, **kwargs)
                    return wrapper
                return decorator
            _decorator = traceable_decorator("process_messages", "chain")
        else:
            _decorator = lambda x: x
        
        @_decorator
        def _inner_process_messages(messages: List[Union[Dict, 'TelegramMessage']], 
                                   config: Optional[RunnableConfig] = None) -> List[Event]:
            messages_dict = [
                EventExtractor._message_to_dict(msg) for msg in messages
            ]

            # Подготовка config для checkpointer, если необходимо
            current_config = config
            if self.checkpointer:
                if current_config and current_config.get("configurable"):
                    # Пользователь предоставил configurable, используем как есть
                    pass
                else:
                    if not current_config:
                        current_config = {}
                    # Создаем временный thread_id на основе хэша сообщений
                    messages_str = json.dumps(messages_dict, sort_keys=True, default=str)
                    thread_hash = hashlib.md5(messages_str.encode()).hexdigest()[:10]
                    current_config = {**current_config, "configurable": {"thread_id": f"temp_{thread_hash}"}}

            initial_state: AgentState = {
                "messages": messages_dict,
                "events": [],
                "processed_count": 0,
                "extraction_errors": [],
                "metadata": {}
            }

            # Выполняем граф
            final_state = self.graph.invoke(initial_state, config=current_config)

            # Логируем результат в LangSmith
            if client:
                try:
                    events_count = len(final_state.get("events", []))
                    client.create_feedback(
                        run_id=None,
                        key="events_extracted",
                        score=min(1.0, events_count / 10.0),  # Нормализованный скор
                        comment=f"Извлечено {events_count} событий"
                    )
                except Exception:
                    pass

            return final_state.get("events", [])
        
        return _inner_process_messages(messages, config)

    def process_messages_batch(
            self,
            messages: List[Union[Dict, 'TelegramMessage']],
            batch_size: int = 10,
            config: Optional[RunnableConfig] = None
    ) -> List[Event]:
        """Пакетная обработка сообщений"""
        if LANGCHAIN_SMITH_AVAILABLE:
            def traceable_decorator(name, run_type):
                def decorator(func):
                    def wrapper(*args, **kwargs):
                        return func(*args, **kwargs)
                    return wrapper
                return decorator
            _decorator = traceable_decorator("process_messages_batch", "chain")
        else:
            _decorator = lambda x: x
        
        @_decorator
        def _inner_process_messages_batch(messages: List[Union[Dict, 'TelegramMessage']],
                                         batch_size: int = 10,
                                         config: Optional[RunnableConfig] = None) -> List[Event]:
            all_events = []

            # Подготовка config для checkpointer, если необходимо
            current_config = config
            if self.checkpointer:
                if current_config and current_config.get("configurable"):
                    # Пользователь предоставил configurable, используем как есть
                    pass
                else:
                    if not current_config:
                        current_config = {}
                    # Создаем временный thread_id для всего батча
                    all_messages_str = json.dumps(messages, sort_keys=True, default=str)
                    thread_hash = hashlib.md5(all_messages_str.encode()).hexdigest()[:10]
                    current_config = {**current_config, "configurable": {"thread_id": f"batch_{thread_hash}"}}

            for i in range(0, len(messages), batch_size):
                batch = messages[i:i + batch_size]
                # Передаем один и тот же config во все вызовы process_messages
                events = self.process_messages(batch, config=current_config)
                all_events.extend(events)

                # Небольшая задержка между батчами
                if i + batch_size < len(messages):
                    time.sleep(0.1)

            return all_events
        
        return _inner_process_messages_batch(messages, batch_size, config)

    def get_graph_info(self) -> Dict:
        """Получение информации о графе"""
        return {
            "nodes": list(self.graph.nodes),
            "edges": list(self.graph.edges),
            "has_checkpointer": self.checkpointer is not None,
            "langsmith_enabled": client is not None
        }


# Фабричная функция для создания агента
def create_event_miner_agent(
        llm,
        model_name: str = "gpt-4o",
        enable_langsmith: bool = True,
        checkpoint_memory: bool = True
) -> EventMinerAgent:
    """
    Создание агента для извлечения событий

    Args:
        llm: Модель LLM (OpenAI, LangChain и т.д.)
        model_name: Название модели
        enable_langsmith: Включить трассировку LangSmith
        checkpoint_memory: Включить чекпоинты памяти

    Returns:
        EventMinerAgent: Настроенный агент
    """
    # Настраиваем LangSmith
    if enable_langsmith and not os.environ.get("LANGSMITH_API_KEY"):
        print("Предупреждение: LANGSMITH_API_KEY не установлен. Трассировка отключена.")

    # Создаем экстрактор и агент
    extractor = EventExtractor(llm, model_name)
    agent = EventMinerAgent(llm, extractor, checkpoint_memory)

    return agent

# Пример использования с РЕАЛЬНЫМ OpenAI и обязательным LangSmith
if __name__ == "__main__":
    import os
    import sys
    
    print("=" * 70)
    print("🚀 EVENT MINER AGENT - РЕАЛЬНЫЙ РЕЖИМ С LANGSMITH")
    print("=" * 70)
    
    # ========== ПРОВЕРКА И УСТАНОВКА ЗАВИСИМОСТЕЙ ==========
    try:
        from langchain_openai import ChatOpenAI
        print("✅ langchain-openai установлен")
    except ImportError:
        print("❌ langchain-openai не установлен")
        print("Устанавливаю...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "langchain-openai"])
        from langchain_openai import ChatOpenAI
        print("✅ langchain-openai установлен")
    
    try:
        from langsmith import Client
        print("✅ langsmith установлен")
    except ImportError:
        print("❌ langsmith не установлен")
        print("Устанавливаю...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "langsmith"])
        from langsmith import Client
        print("✅ langsmith установлен")
    
    # ========== ПРОВЕРКА РЕАЛЬНЫХ КЛЮЧЕЙ ==========
    print("\n🔐 ПРОВЕРКА API КЛЮЧЕЙ:")
    print("-" * 40)
    
    # Получите ключи здесь или установите в переменные окружения
    OPENAI_API_KEY = input("Введите ваш OpenAI API ключ (начинается с sk-): ").strip()
    LANGSMITH_API_KEY = input("Введите ваш LangSmith API ключ (начинается с ls-): ").strip()
    
    if not OPENAI_API_KEY.startswith('sk-'):
        print("❌ НЕВЕРНЫЙ OPENAI КЛЮЧ! Должен начинаться с 'sk-'")
        sys.exit(1)
    
    if not LANGSMITH_API_KEY.startswith('ls-'):
        print("❌ НЕВЕРНЫЙ LANGSMITH КЛЮЧ! Должен начинаться с 'ls-'")
        sys.exit(1)
    
    # Устанавливаем ключи
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
    os.environ["LANGSMITH_TRACING"] = "true"
    os.environ["LANGSMITH_PROJECT"] = "event-miner-agent-production"
    
    print(" Ключи установлены и валидны")
    
    # ========== ИНИЦИАЛИЗАЦИЯ LANGSMITH ==========
    print("\n📊 ИНИЦИАЛИЗАЦИЯ LANGSMITH...")
    try:
        client = Client()
        # Проверяем соединение с LangSmith
        test_project = "event-miner-test-init"
        try:
            client.create_project(project_name=test_project)
            print(f" LangSmith проект '{test_project}' создан")
        except:
            # Проект уже существует - это нормально
            print(f" LangSmith подключен (проект существует)")
        
        # Получаем информацию о пользователе для проверки
        user_info = client.get_user_info()
        print(f" LangSmith аутентифицирован как: {user_info.get('email', 'unknown')}")
        print(" LangSmith трассировка АКТИВИРОВАНА")
        
    except Exception as e:
        print(f" Ошибка инициализации LangSmith: {e}")
        print("Проверьте:")
        print("1. Правильность API ключа LangSmith")
        print("2. Доступ к https://api.smith.langchain.com")
        print("3. Аккаунт LangSmith активирован")
        sys.exit(1)
    
    # ========== СОЗДАНИЕ РЕАЛЬНОЙ LLM ==========
    print("\n СОЗДАНИЕ РЕАЛЬНОЙ LLM МОДЕЛИ...")
    try:
        # Используем gpt-3.5-turbo для экономии в начале
        llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0.3,
            max_tokens=1000,
            timeout=30,
            max_retries=2
        )
        print(" Модель gpt-3.5-turbo создана")
        
        # Тестовый запрос для проверки OpenAI
        print("   Проверяю соединение с OpenAI...")
        test_response = llm.invoke("Ответь 'OK' если ты работаешь")
        if hasattr(test_response, 'content'):
            print("    OpenAI отвечает корректно")
        else:
            print("     Нестандартный ответ от OpenAI")
            
    except Exception as e:
        print(f" Ошибка создания LLM: {e}")
        print("Проверьте:")
        print("1. Правильность OpenAI API ключа")
        print("2. Баланс на счету OpenAI")
        print("3. Доступ к https://api.openai.com")
        sys.exit(1)
    
    # ========== СОЗДАНИЕ АГЕНТА С LANGSMITH ==========
    print("\n СОЗДАНИЕ AGENT С ВКЛЮЧЕННЫМ LANGSMITH...")
    try:
        agent = create_event_miner_agent(
            llm=llm,
            model_name="gpt-3.5-turbo",
            enable_langsmith=True,  #
            checkpoint_memory=True
        )
        print(" Agent создан с включенной трассировкой LangSmith")
        print(f"   • Graph nodes: {len(agent.get_graph_info()['nodes'])}")
        print(f"   • Checkpointer: {'✅' if agent.get_graph_info()['has_checkpointer'] else '❌'}")
        print(f"   • LangSmith: {'✅' if agent.get_graph_info()['langsmith_enabled'] else '❌'}")
        
    except Exception as e:
        print(f" Ошибка создания агента: {e}")
        sys.exit(1)
    
    # ========== ТЕСТОВЫЙ ЗАПРОС ==========
    print("\n" + "=" * 70)
    print(" ВЫПОЛНЕНИЕ ТЕСТОВОГО ЗАПРОСА С LANGSMITH ТРАССИРОВКОЙ")
    print("=" * 70)
    
    test_messages = [
        {
            "id": 1,
            "date": "2024-01-15T10:30:00",
            "text": "Завтра в 15:00 будет онлайн-лекция по машинному обучению для начинающих. Ссылка для подключения: meet.google.com/abc-defg-hij. Приносите ноутбуки."
        },
        {
            "id": 2,
            "date": "2024-01-15T11:00:00",
            "text": "ВАЖНО: Планировочное совещание по проекту 'Новый интерфейс' в среду 17 января в 11:00 в конференц-зале №5 (3 этаж). Будут обсуждаться макеты."
        },
        {
            "id": 3,
            "date": "2024-01-15T12:00:00",
            "text": "Напоминаю про дедлайн сдачи отчетов по кварталу - до пятницы, 19 января, 18:00. Отправлять на report@company.com"
        }
    ]
    
    print(f" Обрабатываю {len(test_messages)} сообщения...")
    print("   (трассировка отправляется в LangSmith в реальном времени)")
    
    try:
        import time
        start_time = time.time()
        
        events = agent.process_messages(test_messages)
        
        elapsed_time = time.time() - start_time
        print(f" Запрос выполнен за {elapsed_time:.2f} сек")
        
        print(f"\n РЕЗУЛЬТАТЫ ИЗВЛЕЧЕНИЯ:")
        print("-" * 50)
        
        if events:
            print(f" Найдено событий: {len(events)}")
            
            for i, event in enumerate(events, 1):
                print(f"\n{i}. [{event.event_type or 'событие'}] {event.title or 'Без названия'}")
                print(f"    Дата: {event.date or 'не указана'} {event.time or ''}")
                print(f"    Место: {event.location or 'не указано'}")
                print(f"    Формат: {' Онлайн' if event.is_online else ' Офлайн' if event.is_online is False else '⚪ Неясно'}")
                if event.description:
                    desc = event.description[:80] + "..." if len(event.description) > 80 else event.description
                    print(f"    Описание: {desc}")
                print(f"    Source ID: {event.source_message_id}")
                
        else:
            print("  Событий не найдено")
            
        print("\n" + "=" * 70)
        print(" LANGSMITH ТРАССИРОВКА АКТИВНА!")
        print("=" * 70)
        print("\nОткройте дашборд LangSmith для просмотра трассировки:")
        print(f" https://smith.langchain.com/o/{client.get_workspace()}/projects/p/event-miner-agent-production")
        print("\nНа дашборде вы увидите:")
        print("• Полный граф выполнения агента")
        print("• Затраты токенов и стоимость запросов")
        print("• Время выполнения каждого узла")
        print("• Извлеченные события")
        print("• Возможные ошибки и предупреждения")
        
    except Exception as e:
        print(f" Ошибка выполнения: {e}")
        print("\nТрассировка ошибки доступна в LangSmith")
        import traceback
        traceback.print_exc()
    
    # ========== ИНСТРУКЦИЯ ДЛЯ ПОЛЬЗОВАТЕЛЯ ==========
    print("\n ИНСТРУКЦИЯ ПО ИСПОЛЬЗОВАНИЮ:")
    print("1. Для обработки своих сообщений вызовите:")
    print("   events = agent.process_messages(ваши_сообщения)")
    print("2. Все вызовы автоматически трассируются в LangSmith")
    print("3. Для пакетной обработки:")
    print("   events = agent.process_messages_batch(сообщения, batch_size=5)")
    print("4. Отслеживайте затраты в OpenAI: https://platform.openai.com/usage")
    print("5. Анализируйте трассировки в LangSmith")


 LangSmith отключен (нет реального ключа)
OpenAI ключ: ❌ Демо или отсутствует
LangSmith ключ: ❌ Демо или отсутствует

Запускаю ТЕСТОВЫЙ пример (без реальных API вызовов)...
Ошибка настройки LangSmith: Failed to POST https://api.smith.langchain.com/sessions in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/sessions', '{"detail":"Forbidden"}')
Тестирую работу агента...
Ошибка при извлечении событий: SimpleTestLLM.invoke() got an unexpected keyword argument 'config'

 Тест пройден! Обработано событий: 0

 Результаты:

Для использования с реальным OpenAI API:
1. Получите ключ на: https://platform.openai.com/api-keys
2. Замените 'sk-ваш-реальный-ключ-здесь' на ваш реальный ключ
3. Убедитесь, что у вас есть средства на счету OpenAI
Тестирую работу агента...
Ошибка при извлечении событий: SimpleTestLLM.invoke() got an unexpected keyword argument 'config'

Тест пройден! Обработано событий: 0
Структура агента полностью работоспособна.

Для использ

In [2]:
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_package("langchain-openai")